# Module 3: Large Scale Circuits and Topology

Scaling from textbook 2-qubit circuits to the 100+ qubit regime required for the competition introduces specific constraints related to topology and mapping.

## 3.1 The Simulation Bottleneck

To train an AI model for QEM, we typically need pairs of $(x_{noisy}, x_{ideal})$.
* **Small Circuits (< 30 qubits):** We can compute $x_{ideal}$ using a statevector simulator.
* **Large Circuits (50+ qubits):** Classical simulation is exponentially hard ($2^{50}$ states). We cannot calculate the ground truth $x_{ideal}$ for arbitrary circuits.

**The Solution:** **Clifford Circuits** and the **Gottesman-Knill Theorem**.
Circuits consisting only of Clifford gates (H, S, CNOT, Pauli) can be simulated in polynomial time $O(N^2)$ on a classical computer, regardless of the number of qubits.

**Clifford Data Regression (CDR):** We train our AI on Clifford circuits (where we know the answer) and assume the learned noise model generalizes to non-Clifford (universal) circuits.

## 3.2 Connectivity and Mapping

Real quantum processors like IBM's Eagle/Osprey follow a **Heavy-Hex lattice** topology. They are not fully connected.

**Constraint:** A CNOT can only be applied between physically connected qubits. If $q_A$ and $q_B$ are not neighbors, the compiler must insert **SWAP** gates.
$$ \text{SWAP}(A, B) = 3 \times \text{CNOT} $$
Routing signals across the chip massively increases circuit depth and error count.

**AI Feature:** Your model should account for the **Topological Distance**. A gate between distant qubits incurs a penalty proportional to the path length.

## 3.3 Implementation: Generating Large-Scale Training Data

We will create a dataset generator that produces random **Clifford circuits**. These serve as the "training ground" for your AI, allowing it to learn the noise characteristics of a 100-qubit device without needing a supercomputer.

In [2]:
import numpy as np
import random
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Clifford
from qiskit_aer import AerSimulator

def create_random_clifford_circuit(num_qubits, depth):
    """
    Manually builds a random circuit using ONLY Clifford gates (H, S, CX, X, Y, Z).
    This guarantees compatibility with the Clifford simulator.
    """
    qc = QuantumCircuit(num_qubits)
    clifford_gates_1q = ['h', 's', 'x', 'y', 'z']

    for _ in range(depth):
        # Pick a random qubit
        q = random.randint(0, num_qubits - 1)

        if num_qubits > 1 and random.random() > 0.5:
            # 50% chance of 2-qubit gate (CX)
            target = random.randint(0, num_qubits - 1)
            while target == q: # Ensure target is different
                target = random.randint(0, num_qubits - 1)
            qc.cx(q, target)
        else:
            # 50% chance of 1-qubit gate
            gate = random.choice(clifford_gates_1q)
            getattr(qc, gate)(q)

    return qc

def generate_clifford_data(n_qubits=5, depth=10, n_samples=3):
    """
    Generates dataset using the safe random Clifford generator.
    """
    dataset = []

    for i in range(n_samples):
        # 1. Use our SAFE generator
        qc = create_random_clifford_circuit(n_qubits, depth)

        # 2. Calculate "Ideal" value efficiently using Clifford simulator
        # We assume the input state is |0...0>
        cliff = Clifford(qc)

        # Calculate Expectation <Z> for qubit 0
        # We can get this directly from the Stabilizer Table without simulation shots!
        # The Clifford object tracks the evolution of Pauli Z.
        # If Z_0 evolves to +Z, exp=1. If -Z, exp=-1. If X/Y, exp=0.

        # For simplicity in this demo, we'll use the Aer Stabilizer simulator
        # which is optimized for this.
        qc_sim = qc.copy()
        qc_sim.measure_all()
        sim_ideal = AerSimulator(method='stabilizer')
        result_ideal = sim_ideal.run(qc_sim, shots=1000).result()
        counts_ideal = result_ideal.get_counts()

        # Calc <Z>
        shots = sum(counts_ideal.values())
        p0 = sum(v for k, v in counts_ideal.items() if k.endswith('0')) / shots
        p1 = sum(v for k, v in counts_ideal.items() if k.endswith('1')) / shots
        ideal_z = p0 - p1

        dataset.append({
            'circuit_index': i,
            'n_qubits': n_qubits,
            'depth': depth,
            'ideal_expectation': ideal_z,
            'circuit_obj': qc
        })

    return dataset

# --- Generate Sample Data ---
data = generate_clifford_data(n_qubits=5, depth=10, n_samples=3)
print(f"Generated {len(data)} Clifford training samples.")
print(f"Sample 0 Ideal <Z>: {data[0]['ideal_expectation']}")

Generated 3 Clifford training samples.
Sample 0 Ideal <Z>: 1.0
